In [ ]:
import os,sys
sys.path.insert(0,'..')

import requests
import json
from datetime import datetime, timezone, timedelta
import os
from time import time
from backend.constants.constants import ACCESS_TOKEN
from time import sleep
import pytz


t1 = time()
path_json = '/home/alberto/Docker/Trading/analysis/json/data.json'
new_datetime_start = datetime.now(timezone.utc)
print('Script started at: ', new_datetime_start)
DAYS=1 # how many days for retrieving data

#ENDPOINT = 'https://algocrypto.eu'
ENDPOINT = 'http://localhost'
method = '/analysis/get-data/'
datetime_start = datetime(2023,3,31)
datetime_start = pytz.utc.localize(datetime_start)

datetime_start_iso = datetime_start.isoformat()



headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}", 
        "Content-Type": "application/json"
    }

# if data.json already exists, get saved data
if os.path.isfile('./json/data.json'):
    # read file
    
    f = open ('./json/data.json', "r")
    data = json.loads(f.read())
            
    datetime_start_iso = data['datetime_creation']
    date_split = datetime_start_iso.split('T')
    date = date_split[0]
    hour = date_split[1].split('.')[0]

    datetime_start = datetime.fromisoformat(datetime_start_iso)
    datetime_end_iso = (datetime_start + timedelta(days=DAYS)).isoformat()
    
#if data.json does not exists, initialize data variable
else:
    date_split = datetime_start_iso.split('T')
    date = date_split[0]
    hour = date_split[1].split('.')[0]
    datetime_end_iso = (datetime_start + timedelta(days=DAYS)).isoformat()
    data = {'datetime_creation': datetime_start_iso, 'data': {}}
    
    #print(new_datetime_start - datetime_start)

while (new_datetime_start - datetime_start).days >= 0:
    while datetime.now().second != 15:
        sleep(0.9)
    
    days_timedelta_iteration = (new_datetime_start - datetime_start).days
    date_split = datetime_start_iso.split('T')
    date = date_split[0]
    hour = date_split[1].split('.')[0]

    if days_timedelta_iteration == 0:
        datetime_creation = datetime.now(timezone.utc).isoformat()
    else:
        datetime_creation = datetime_end_iso

    # UPDATE DATETIME_CREATION
    data['datetime_creation'] = datetime_creation
    
    params = {'datetime_start': datetime_start_iso, 'datetime_end': datetime_end_iso}
    url = ENDPOINT + method
    print(f'Making the request to {url}')
    response = requests.get(url, headers=headers, params=params)
    new_instrument_data=0

    if response.status_code == 200:
        new_data = response.json()
        print(f'Starting Time: {date} {hour}')
        n_instrument_newdata = 0
        n_observations = 0
        # UPDATE DATA
        for instrument_name in new_data:
            if instrument_name in data['data']:
                for trade in new_data[instrument_name]:
                    data['data'][instrument_name].append(trade)
            else:
                print(f'new instrument_name: {instrument_name}')
                new_instrument_data += 1
                data['data'][instrument_name] = []
                for trade in new_data[instrument_name]:
                    data['data'][instrument_name].append(trade)


        print(f'Iterationg through new instrument is completed.')
        if new_instrument_data != 0:
            print(f'{new_instrument_data} of the instrument fetched are NEW')
        else:
            print("No New Instrument")
        btc_obs = len(new_data['BTCUSDT'])
        print(f'{btc_obs} new observations for Bitcoin and others not new')
        
        #data['datetime_creation'] = datetime_creation
        #data = {'datetime_creation': datetime_creation, 'data': data}

        json_string = json.dumps(data)
        with open(path_json, 'w') as outfile:
            outfile.write(json_string)
        print(f'new Json saved in {path_json}')

        datetime_start = datetime_start + timedelta(days=DAYS)
        datetime_end = datetime_start + timedelta(days=DAYS)
        datetime_start_iso = datetime_start.isoformat()
        datetime_end_iso = datetime_end.isoformat()
        new_datetime_start = datetime.now(timezone.utc)

    else:
        print('SOMETHING WRONG HAPPENED: ', response.status_code)
        print()

    
t2 = time()
t = t2-t1
print(f'Time Spent: {t} ')